In [ ]:
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from skimage.io import imread, imshow
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

In [ ]:
#Load test lable here.
open("dataset/Eval/test_label.json")
json_gt = [json.loads(line) for line in open("dataset/Eval/test_label.json")]

In [ ]:
#You will need to load a model under the name "model". As long as model.predict(img) return an image, then the it should work

In [ ]:
#load image path. Note that the json file provide the image path for each clip, so make sure you get the right outer path
image_path=["dataset/Eval/"+json_gt[i]['raw_file'] for i in range(0,len(json_gt))]
imshow(images)
plt.show()

In [ ]:
#The module postprocessing(pic,tol_dis,tol_sharp) post process. "pic" is the index in image_path. "tol_dis" is the tolerant distance for smooth curve (recommend setting it to 1000)."tol_sharp" denote how sharp the lane can turn.
#The module return an array of positive test and an array of negtive test.Each array has 4 elements, represents the accuracy of each threshold 25,50,75,100

def isClosed(lane1,lane2,threshold):
    sum=0
    count=0
    for i in range(0,len(lane1)):
        if lane1[i]!=-2 and lane2[i]!=-2:
            sum=abs(lane1[i]-lane2[i])+sum
            count=count+1
    if count==0:
        return False
    elif sum/count>threshold:
        return False
    else:
        return True
def evaluate(lane,json,threshold):
    oldcor=0
    oldreq=0
    for i in range(0,len(json)):
        correct=0
        request=0
        for j in range(0,56):
            if lane[j]!=-2:
                request=request+1
                if abs(lane[j]-json[i][j])<threshold or ((lane[j]<threshold or lane[j]>1280-threshold) and json[i][j]==-2):
                    correct=correct+1
        if (oldreq==0) or (request>0 and correct/request>oldcor/oldreq):
            oldcor=correct
            oldreq=request
    return oldcor,oldreq
def near_smooth(coord2,coord1,coord3,tol_dis,tol_convex,skip_convex):
    y1=coord1[0]
    y2=coord2[0]
    y3=coord3[0]
    x1=coord1[1]
    x2=coord2[1]
    x3=coord3[1]
    distance=(y1-y2)**2+(x1-x2)**2
    if not skip_convex:
        m1=(x1-x2)/(y1-y2)
        m2=(x3-x2)/(y3-y2)
        convex= abs(m2-m1)/abs(y1-y3)
        #print(coord1,coord2,coord3)
        #print("m1=",m1,", m2=",m2,", y1=",y1,", y3=",y3,", convex=",convex)
        if np.amin(distance)>tol_dis and np.amin(convex)>tol_convex:
            return False
        else:
            return True
    elif distance>tol_dis:
        return False
    else:
        return True
def postprocessing(pic,tol_dis,tol_sharp):
    images=cv2.resize(imread(image_path[pic],0), dsize=(128, 128), interpolation=cv2.INTER_NEAREST)
    img = images.reshape(1,128,128,3)
    prediction=np.squeeze(model.predict(img))
    prediction = cv2.resize(prediction, dsize=(1280, 720), interpolation=cv2.INTER_NEAREST)
    tol_dis=tol_dis
    tol_invtan=tol_sharp
    oldlane=np.array([])
    toreq=np.zeros(4)
    tocorr=np.zeros(4)
    pp=np.array([])
    tocorrneg=np.zeros(4)
    toreqneg=np.zeros(4)
    for j in range(1,6):
        new_pred=np.round(prediction[:,:,j],decimals=3)
        group1=np.array([])
        group2=np.array([])
        predim1=np.zeros([720,1280])
        predim2=np.zeros([720,1280])
        if np.average(new_pred[np.nonzero(new_pred)])<.01:
            continue
        threshold=np.average(new_pred[np.nonzero(new_pred)])+np.std(new_pred[np.nonzero(new_pred)])
        for y_coord in range(160,720):
            row=prediction[y_coord,:,j]
            prob=np.amax(row)       
            if prob>threshold:
                x_coord=np.amin(np.squeeze(np.where(row == prob)))
                predim1[y_coord,x_coord]=1
            else:
                continue
            group1add=group1.size==0 or (group1.size==2 and near_smooth(group1[0],[y_coord,x_coord],group1[-1],tol_dis,0,True)) or (group1.size>2 and near_smooth(group1[-1],[y_coord,x_coord],group1[-2],tol_dis,tol_invtan,False))
            group2add=group2.size==0 or (group2.size==2 and near_smooth(group2[0],[y_coord,x_coord],group2[-1],tol_dis,0,True)) or (group2.size>2 and near_smooth(group2[-1],[y_coord,x_coord],group2[-2],tol_dis,tol_invtan,False))
            if group1.size>0 and group2.size>0 and group1add and group2add:
                group1=np.reshape(np.append(group1,[y_coord,x_coord]),(-1,2))
                group2=np.reshape(np.append(group2,[y_coord,x_coord]),(-1,2))
            elif group1add:
                group1=np.reshape(np.append(group1,[y_coord,x_coord]),(-1,2))
            elif group1.size>0 and group2add:
                group2=np.reshape(np.append(group2,[y_coord,x_coord]),(-1,2))
            else:
                if group1.size<=group2.size:
                    group1=group2
                group2=np.array([[y_coord,x_coord]])
        if group1.size<=group2.size:
            group1=group2
        group1=group1.astype('int64')
        if group1.size<50 :
            continue
        if group1[-1,0]<715:
            t=group1[-52:-1,0]
            xn=group1[-52:-1,1]
            f = np.poly1d(np.polyfit(t, xn, 1))
            for i in range(round(group1[-1,0])+1,715):
                guess=round(f(i))
                if 0<guess<1280:
                     group1=np.reshape(np.append(group1,[i,guess],),(-1,2))
                else:
                    break
        high=group1[0,0]
        for i in range (high,min(710,group1[-1,0])-2):
            if group1[i-high,0]!=i:
                t=group1[i-high-1:i-high+1,0]
                xn=group1[i-high-1:i-high+1,1]
                f = np.poly1d(np.polyfit(t, xn, 1))
                guess=[[i,round(f(i))]for i in range(i,group1[i-high,0])]
                group1=np.insert(group1,i-high,guess,axis=0)
            predim2[group1[i-high,0],group1[i-high,1]]=j
            predim2=predim2.astype('uint8')
        lane=np.array([])
        for y in range(160,720,10):
            if y<group1[0][0]:
                lane=np.append(lane,[-2])
            elif y>group1[-1][0]:
                lane=np.append(lane,[-2])
            else:
                lane=np.append(lane,np.squeeze(group1[np.where(y==group1[:,0]),1]))
        if len(oldlane)>0 and isClosed(oldlane,lane,50):
            continue
        oldlane=lane
        pp=np.append(pp,lane)
        for i in range(0,4):            
            correct,request=evaluate(lane,json_gt[pic]['lanes'],i*25+25)
            tocorr[i]=tocorr[i]+correct
            toreq[i]=toreq[i]+request
    pp=pp.reshape(-1,56)
    for j in range(0,len(json_gt[pic]['lanes'])):
        for i in range(0,4):            
            correct,request=evaluate(json_gt[pic]['lanes'][j],pp,i*25+25)
            tocorrneg[i]=tocorrneg[i]+correct
            toreqneg[i]=toreqneg[i]+request
    evalpos=np.zeros(4)
    evalneg=np.zeros(4)
    for i in range(0,4):
        if toreq[i]!=0:
            evalpos[i]=tocorr[i]/toreq[i]
        if toreqneg[i]!=0:
            evalneg[i]=tocorrneg[i]/toreqneg[i]
    return evalpos,evalneg

In [ ]:
#Evaluate all the test set. picper is an array of positive test result of each picture. Each picture should have an array of 4 element, represent result of each threshould. picperneg is the array for negative test result
aveper=np.zeros(4)
avenegper=np.zeros(4)
picper=np.array([])
picperneg=np.array([])
count=0
for pic in range(0,len(json_gt)):
    try:
        per,perneg=postprocessing(pic,5000,10**(-5))
        aveper=aveper+per
        avenegper=avenegper+perneg
        count=count+1
        if pic%100==0:
            print(pic)
        picper=np.append(picper,per)
        picperneg=np.append(picperneg,perneg)
    except:
        print("error in pic", pic)
        picper=np.append(picper,np.zeros(4))
        picperneg=np.append(picperneg,np.zeros(4))
picper=picper.reshape(-1,4)
picperneg=picperneg.reshape(-1,4)